In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import collections
import cv2
import datetime
import math
import numpy
import os
import pathlib
import skimage.io
import skimage.transform
import sklearn.model_selection
import tensorflow
import time

from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma
)
from IPython.display import Markdown as md
from markdownTable import markdownTable

from files import create_folder, save_fit_history, save_lossgraph, save_figs
from metrics import dice_coef, jaccard_distance, jaccard_distance_loss
from model import evaluate, unet_model
from AugmentationSequence import AugmentationSequence

# GPU

In [2]:
gpus = tensorflow.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f"GPU: {gpu.name}")
            tensorflow.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

GPU: /physical_device:GPU:0


In [9]:
cfg = {
    "channel": 3,
    "batch_size": 4,
    "fold": 5,
    "epochs": 75,
    "image_size": 400,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_dataset": "dataset",
    "path_out": "out"
}
images_folder = os.path.join(cfg["path_dataset"], "original")
masks_folder = os.path.join(cfg["path_dataset"], "mask")

In [10]:
list_labels = list([])
list_images = list([])
list_images_names = list([])
for file in pathlib.Path(masks_folder).rglob("*"):
    mask = skimage.io.imread(str(file.resolve()))
    mask = numpy.float32(mask > 200)
    list_labels.append(mask)

    image = skimage.io.imread(os.path.join(images_folder, file.name))
    image = skimage.transform.resize(image, (cfg["image_size"], cfg["image_size"]), anti_aliasing=True)
    image = numpy.float32(image)
    list_images.append(image)

    list_images_names.append(str(file.stem))

print(len(list_labels), len(list_images), len(list_images_names))

375 375 375


In [11]:
x = numpy.array(list_images).reshape((len(list_images), cfg["image_size"], cfg["image_size"], cfg["channel"]))
y = numpy.array(list_labels).reshape((len(list_labels), cfg["image_size"], cfg["image_size"], 1))

print(x.shape, y.shape)

(375, 400, 400, 3) (375, 400, 400, 1)


In [12]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
l = list([])
for (train_index, test_index) in kf.split(x):
    l = l + test_index.tolist()
print(len(list(set(l))))
print(collections.Counter(sorted(l)))

375
Counter({0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1, 100: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 1, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 1, 130: 1, 131: 1, 132: 1, 133: 1, 134: 1, 135: 1, 136: 1, 1

In [13]:
from files import save_lossgraph

models = []
list_evaluate = list([])
current_datetime = datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
path = os.path.join(cfg["path_out"], current_datetime)
create_folder(list([path]))
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg["val_size"], random_state=cfg["random_state"])

    print(x_train.shape)
    print(x_val.shape)
    print(x_test.shape)
    print(x.shape)

    path_fold = os.path.join(path, str(fold))
    create_folder(list([path_fold]))

    augment = Compose([
        HorizontalFlip(),
        ShiftScaleRotate(rotate_limit=45, border_mode=cv2.BORDER_CONSTANT),
        ElasticTransform(border_mode=cv2.BORDER_CONSTANT),
        RandomBrightness(),
        RandomContrast(),
        RandomGamma()
    ])
    steps_per_epoch = math.ceil(x_train.shape[0] / cfg["batch_size"])
    train_generator = AugmentationSequence(x_train, y_train, cfg["batch_size"], augment)
    reduce_learning_rate = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=1)
    filename_model = os.path.join(path_fold, "unet.h5")
    checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
    strategy = tensorflow.distribute.MirroredStrategy()

    with strategy.scope():
        model = unet_model(cfg)
        adam_opt = tensorflow.keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
        model.compile(optimizer=adam_opt, loss=jaccard_distance_loss,
                      metrics=[dice_coef, jaccard_distance, tensorflow.keras.metrics.Precision(),
                               tensorflow.keras.metrics.Recall()])

    tensorflow.keras.backend.clear_session()
    start_time = time.time()
    fit = model.fit(train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=cfg["epochs"],
              validation_data=(x_val, y_val),
               callbacks=[checkpointer, reduce_learning_rate]
              )
    end_time = time.time() - start_time

    save_fit_history(fold, fit, path_fold)
    save_lossgraph(fold, fit, path_fold)
    list_evaluate.append(evaluate(end_time, fold, model, x_train, x_val, x_test, y_train, y_val, y_test))

    models.append(model)
    # model = tensorflow.keras.models.load_model("unet.h5", custom_objects = {"jaccard_distance_loss": jaccard_distance_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })

    save_figs(list_images_names, model, path_fold, x)
    break

(285, 400, 400, 3)
(15, 400, 400, 3)
(75, 400, 400, 3)
(375, 400, 400, 3)


C:\Users\xandao\anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1613: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
C:\Users\xandao\anaconda3\lib\site-packages\albumentations\augmentations\transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Epoch 1/2
36/36 [==============================] - ETA: 0s - loss: 0.6447 - dice_coef: 0.5173 - jaccard_distance: 0.3541 - precision: 0.5636 - recall: 0.9506
Epoch 1: val_loss improved from inf to 0.80858, saving model to out\05-08-2022-21-06-02\0\unet.h5
36/36 [==============================] - 231s 6s/step - loss: 0.6447 - dice_coef: 0.5173 - jaccard_distance: 0.3541 - precision: 0.5636 - recall: 0.9506 - val_loss: 0.8086 - val_dice_coef: 0.3213 - val_jaccard_distance: 0.1914 - val_precision: 0.6581 - val_recall: 0.1214 - lr: 0.0010
Epoch 2/2
36/36 [==============================] - ETA: 0s - loss: 0.5393 - dice_coef: 0.6234 - jaccard_distance: 0.4587 - precision: 0.7385 - recall: 0.9720
Epoch 2: val_loss improved from 0.80858 to 0.72885, saving model to out\05-08-2022-21-06-02\0\unet.h5
36/36 [==============================] - 160s 4s/step - loss: 0.5393 - dice_coef: 0.6234 - jaccar

In [16]:
filename_cfg = os.path.join(path, "cfg.md")
with open(filename_cfg, "w") as file:
    file.write(markdownTable(list([cfg])).getMarkdown())
    file.close()

md(markdownTable(list([cfg])).getMarkdown())

```
+-------------------------------------------------------------------------------------------------------------+
|channel|batch_size|fold|epochs|image_size|learning_rate|random_state|test_size|val_size|path_dataset|path_out|
+-------+----------+----+------+----------+-------------+------------+---------+--------+------------+--------+
|   3   |     8    |  5 |   2  |    400   |     0.0     |    1234    |   0.2   |  0.05  |   dataset  |   out  |
+-------------------------------------------------------------------------------------------------------------+```

In [18]:
mean_std = {
    "mean_loss_train": numpy.mean(list([evaluate["loss_train"] for evaluate in list_evaluate])),
    "std_loss_train": numpy.std(list([evaluate["loss_train"] for evaluate in list_evaluate])),
    "mean_dice_train": numpy.mean(list([evaluate["dice_train"] for evaluate in list_evaluate])),
    "std_dice_train": numpy.std(list([evaluate["dice_train"] for evaluate in list_evaluate])),
    "mean_jaccard_train": numpy.mean(list([evaluate["jaccard_train"] for evaluate in list_evaluate])),
    "std_jaccard_train": numpy.std(list([evaluate["jaccard_train"] for evaluate in list_evaluate])),
    "mean_precision_train": numpy.mean(list([evaluate["precision_train"] for evaluate in list_evaluate])),
    "std_precision_train": numpy.std(list([evaluate["precision_train"] for evaluate in list_evaluate])),
    "mean_recall_train": numpy.mean(list([evaluate["recall_train"] for evaluate in list_evaluate])),
    "std_recall_train": numpy.std(list([evaluate["recall_train"] for evaluate in list_evaluate])),
    "mean_loss_val": numpy.mean(list([evaluate["loss_val"] for evaluate in list_evaluate])),
    "std_loss_val": numpy.std(list([evaluate["loss_val"] for evaluate in list_evaluate])),
    "mean_dice_val": numpy.mean(list([evaluate["dice_val"] for evaluate in list_evaluate])),
    "std_dice_val": numpy.std(list([evaluate["dice_val"] for evaluate in list_evaluate])),
    "mean_jaccard_val": numpy.mean(list([evaluate["jaccard_val"] for evaluate in list_evaluate])),
    "std_jaccard_val": numpy.std(list([evaluate["jaccard_val"] for evaluate in list_evaluate])),
    "mean_precision_val": numpy.mean(list([evaluate["precision_val"] for evaluate in list_evaluate])),
    "std_precision_val": numpy.std(list([evaluate["precision_val"] for evaluate in list_evaluate])),
    "mean_recall_val": numpy.mean(list([evaluate["recall_val"] for evaluate in list_evaluate])),
    "std_recall_val": numpy.std(list([evaluate["recall_val"] for evaluate in list_evaluate])),
    "mean_loss_test": numpy.mean(list([evaluate["loss_test"] for evaluate in list_evaluate])),
    "std_loss_test": numpy.std(list([evaluate["loss_test"] for evaluate in list_evaluate])),
    "mean_dice_test": numpy.mean(list([evaluate["dice_test"] for evaluate in list_evaluate])),
    "std_dice_test": numpy.std(list([evaluate["dice_test"] for evaluate in list_evaluate])),
    "mean_jaccard_test": numpy.mean(list([evaluate["jaccard_test"] for evaluate in list_evaluate])),
    "std_jaccard_test": numpy.std(list([evaluate["jaccard_test"] for evaluate in list_evaluate])),
    "mean_precision_test": numpy.mean(list([evaluate["precision_test"] for evaluate in list_evaluate])),
    "std_precision_test": numpy.std(list([evaluate["precision_test"] for evaluate in list_evaluate])),
    "mean_recall_test": numpy.mean(list([evaluate["recall_test"] for evaluate in list_evaluate])),
    "std_recall_test": numpy.std(list([evaluate["recall_test"] for evaluate in list_evaluate]))
}

filename_mean = os.path.join(path, "mean.md")
with open(filename_mean, "w") as file:
    file.write(markdownTable(list([mean_std])).getMarkdown())
    file.close()

md(markdownTable(list([mean_std])).getMarkdown())

```
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|  mean_loss_train |std_loss_train|  mean_dice_train |std_dice_train| mean_jaccard_train|std_jaccard_train|mean_precision_train|std_precision_train| mean_recall_train |std_recall_train|   mean_loss_val  |std_loss_val|   mean_dice_val  |std_dice_val| mean_jaccard_val |std_jaccard_val|mean_precision_val|std_precision_val|  mean_recall_val  |std_recall_val|  mean_loss_test  |std_loss_test|  mean_dice_test  |std_dice_test| mean_jaccard_test |std_jaccard_test|mean_precision_test|std_precision_test| mean_recall_test|std_recall_test|
+------------------+--------------+------------------+--------------+-------------------+-----------------+--------------------+-------------------+-------------------+----------------+------------------+------------+------------------+------------+------------------+---------------+------------------+-----------------+-------------------+--------------+------------------+-------------+------------------+-------------+-------------------+----------------+-------------------+------------------+-----------------+---------------+
|0.7528406381607056|      0.0     |0.3956473469734192|      0.0     |0.24693772196769714|       0.0       | 0.6840612888336182 |        0.0        |0.35886964201927185|       0.0      |0.7288548946380615|     0.0    |0.4266149699687958|     0.0    |0.2711450755596161|      0.0      |0.6685186624526978|       0.0       |0.48654139041900635|      0.0     |0.7784258127212524|     0.0     |0.3635934293270111|     0.0     |0.22287869453430176|       0.0      | 0.6381967663764954|        0.0       |0.376807302236557|      0.0      |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+```

In [21]:
for evaluate in list_evaluate:
    filename_fold = os.path.join(path, str(evaluate["fold"]), "metrics.md")
    with open(filename_fold, "w") as file:
        file.write(markdownTable(list([evaluate])).getMarkdown())
        file.close()